# Proprocessing dataset

In [2]:
import pandas as pd
import datasets
from datasets import load_dataset, Dataset, DatasetDict

Qdata_frame = pd.read_csv('../dataset/train3.csv')


In [3]:
# labelling
Qdata_frame['topic'] = Qdata_frame['topic'].apply(lambda x: x.strip().split(', '))
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Qdf = Qdata_frame.join(pd.DataFrame(mlb.fit_transform(Qdata_frame['topic']), columns=mlb.classes_))

In [4]:
Qdf.to_csv(r'../dataset/train3_clean.csv')

In [5]:
Qdata_Cleanframe = pd.read_csv('../dataset/train3_clean.csv')

In [43]:
max(Qdata_Cleanframe['String'].map(lambda x: len(x.split(' '))))

63

In [61]:
# split the train, test, and validation data
import sklearn
from sklearn.model_selection import train_test_split

Qdf_train, Qdf_remain = train_test_split(Qdata_Cleanframe, train_size=0.8)
Qdf_valid, Qdf_test = train_test_split(Qdf_remain, test_size=0.5)

In [62]:
len(Qdf_train.columns)

30

In [63]:
len(Qdf_valid.columns)

30

In [64]:
len(Qdf_test.columns)

30

In [65]:
# change to hugging face dataset type
trainDS = Dataset.from_pandas(Qdf_train)
validDS = Dataset.from_pandas(Qdf_valid)
testDS = Dataset.from_pandas(Qdf_test)
ds = DatasetDict()
ds["train"] = trainDS
ds["valid"] = validDS
ds["test"] = testDS
print(ds)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'String', 'topic', 'type', 'Algorithm Design', 'Communication Software and Protocols', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Database', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Input and Output Devices', 'Intellectual Property', 'Internet Services and Applications', 'Introduction to HTML', 'Introduction to Information Processing', 'Multimedia Elements', 'Number System', 'Problem- Formulation and Analysis', 'Program Development', 'Program Testing and Debugging', 'Secondary Storage Devices', 'Spreadsheet', 'System Software', 'System Unit of a Computer System', 'The Networking and Internet Basics', 'Threats and Security on the Internet', 'Word Presentation', '__index_level_0__'],
        num_rows: 342
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'id', 'String', 'topic', 'type', 'Algorithm Design', 'Communication Software and Proto

In [66]:
#  clear not used columns
columns = ds.column_names
columns_toRemove = ["id", "type", "topic"]
new_ds = ds.remove_columns(columns_toRemove)
print(new_ds)


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Communication Software and Protocols', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Database', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Input and Output Devices', 'Intellectual Property', 'Internet Services and Applications', 'Introduction to HTML', 'Introduction to Information Processing', 'Multimedia Elements', 'Number System', 'Problem- Formulation and Analysis', 'Program Development', 'Program Testing and Debugging', 'Secondary Storage Devices', 'Spreadsheet', 'System Software', 'System Unit of a Computer System', 'The Networking and Internet Basics', 'Threats and Security on the Internet', 'Word Presentation', '__index_level_0__'],
        num_rows: 342
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Communication Software and Protocols', 'Computer System', 'Data Manipulation a

In [67]:
labels = [label for label in new_ds['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [68]:
# extract the labels
labels
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [69]:
labels

['Algorithm Design',
 'Communication Software and Protocols',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Database',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Input and Output Devices',
 'Intellectual Property',
 'Internet Services and Applications',
 'Introduction to HTML',
 'Introduction to Information Processing',
 'Multimedia Elements',
 'Number System',
 'Problem- Formulation and Analysis',
 'Program Development',
 'Program Testing and Debugging',
 'Secondary Storage Devices',
 'Spreadsheet',
 'System Software',
 'System Unit of a Computer System',
 'The Networking and Internet Basics',
 'Threats and Security on the Internet',
 'Word Presentation']

In [70]:
from transformers import AutoTokenizer, BertForSequenceClassification
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\user/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\vocab.txt

In [71]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in labels}
    labels_matrix = np.zeros((len(text), len(labels)))
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [72]:
encoded_ds = new_ds.map(encode_data, batched=True, remove_columns=new_ds['train'].column_names)
encoded_ds['train'][0]
encoded_ds['train'][0].keys()

100%|██████████| 1/1 [00:00<00:00, 167.04ba/s]


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [73]:
encoded_ds['train']['input_ids'][0]

[101,
 2302,
 2035,
 27483,
 2062,
 5527,
 4496,
 11243,
 13379,
 1010,
 6592,
 2048,
 4725,
 2000,
 5547,
 1996,
 5371,
 2946,
 1997,
 1996,
 7760,
 1012,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [74]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Computer System']

# Model training

In [75]:
# TODO:  tokenize text, feature extraction, multilabel classification
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [76]:
from transformers import AutoModelForSequenceClassification
# define model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", problem_type="multi_label_classification", num_labels=len(labels), id2label=id2label, label2id=label2id)
model.cuda()

loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Algorithm Design",
    "1": "Communication Software and Protocols",
    "2": "Computer System",
    "3": "Data Manipulation and Analysis",
    "4": "Data Organisation and Data Control",
    "5": "Database",
    "6": "Elementary Web Authoring",
    "7": "Health and Ethical Issues",
    "8": "Input and Output Devices",
    "9": "Intellectual Property",
    "10": "Internet Services and Applications",
    "11": "Introduction to HTML",
    "12": "Introduction to Information Processing",
    "13": "Mul

In [77]:
# train model
batch_size = 8
metric_name = "f1"

In [78]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"nbroad/ESG-BERT",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [79]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [80]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [81]:
encoded_ds['train']['input_ids'][0]

tensor([  101,  2302,  2035, 27483,  2062,  5527,  4496, 11243, 13379,  1010,
         6592,  2048,  4725,  2000,  5547,  1996,  5371,  2946,  1997,  1996,
         7760,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [82]:
#forward pass
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6953, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.2095,  0.2323, -0.1532,  0.1598, -0.1179, -0.3531, -0.2512,  0.6429,
          0.7513, -0.0927,  0.0568, -0.4112,  0.1029, -0.2111,  0.1010,  0.1352,
         -0.1768, -0.1278, -0.1133,  0.1024, -0.0047,  0.0091, -0.4164, -0.1585,
         -0.2920]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [83]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [84]:
trainer.train()

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 342
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1290
  Number of trainable parameters = 109501465
  3%|▎         | 42/1290 [00:03<01:56, 10.73it/s]***** Running Evaluation *****
  Num examples = 43
  Batch size = 8

  3%|▎         | 43/1290 [00:04<01:56, 10.73it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-43
Configuration saved in nbroad/ESG-BERT\checkpoint-43\config.json


{'eval_loss': 0.3255498707294464, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.1348, 'eval_samples_per_second': 319.07, 'eval_steps_per_second': 44.521, 'epoch': 1.0}


Model weights saved in nbroad/ESG-BERT\checkpoint-43\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-43\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-43\special_tokens_map.json
  7%|▋         | 85/1290 [00:13<01:52, 10.69it/s]***** Running Evaluation *****
  Num examples = 43
  Batch size = 8

  7%|▋         | 86/1290 [00:13<01:52, 10.69it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-86
Configuration saved in nbroad/ESG-BERT\checkpoint-86\config.json


{'eval_loss': 0.22899845242500305, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.1342, 'eval_samples_per_second': 320.325, 'eval_steps_per_second': 44.696, 'epoch': 2.0}


Model weights saved in nbroad/ESG-BERT\checkpoint-86\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-86\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-86\special_tokens_map.json
 10%|█         | 129/1290 [00:19<01:46, 10.89it/s]***** Running Evaluation *****
  Num examples = 43
  Batch size = 8

 10%|█         | 129/1290 [00:19<01:46, 10.89it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-129
Configuration saved in nbroad/ESG-BERT\checkpoint-129\config.json


...

Model weights saved in nbroad/ESG-BERT\checkpoint-1247\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-1247\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-1247\special_tokens_map.json
100%|█████████▉| 1289/1290 [04:39<00:00, 10.88it/s]***** Running Evaluation *****
  Num examples = 43
  Batch size = 8

100%|██████████| 1290/1290 [04:39<00:00, 10.88it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-1290
Configuration saved in nbroad/ESG-BERT\checkpoint-1290\config.json


{'eval_loss': 0.10413876175880432, 'eval_f1': 0.5633802816901408, 'eval_roc_auc': 0.6995121951219512, 'eval_accuracy': 0.3488372093023256, 'eval_runtime': 0.1368, 'eval_samples_per_second': 314.414, 'eval_steps_per_second': 43.872, 'epoch': 30.0}


Model weights saved in nbroad/ESG-BERT\checkpoint-1290\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-1290\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-1290\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from nbroad/ESG-BERT\checkpoint-1204 (score: 0.5833333333333334).
100%|██████████| 1290/1290 [04:48<00:00,  4.47it/s]

{'train_runtime': 288.7256, 'train_samples_per_second': 35.535, 'train_steps_per_second': 4.468, 'train_loss': 0.13947496488112812, 'epoch': 30.0}


TrainOutput(global_step=1290, training_loss=0.13947496488112812, metrics={'train_runtime': 288.7256, 'train_samples_per_second': 35.535, 'train_steps_per_second': 4.468, 'train_loss': 0.13947496488112812, 'epoch': 30.0})

In [85]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 43
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 60.08it/s]


{'eval_loss': 0.10414627939462662,
 'eval_f1': 0.5833333333333334,
 'eval_roc_auc': 0.7095121951219512,
 'eval_accuracy': 0.37209302325581395,
 'eval_runtime': 0.1268,
 'eval_samples_per_second': 339.117,
 'eval_steps_per_second': 47.319,
 'epoch': 30.0}

In [95]:
text = "ALGI and ALG3 below should always produce the same result. Complete Line 3 below."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}


outputs = trainer.model(**encoding)
# outputs = trainer.model(encoded_ds["test"])

In [96]:
encoding

{'input_ids': tensor([[ 101, 2632, 5856, 1998, 2632, 2290, 2509, 2917, 2323, 2467, 3965, 1996,
          2168, 2765, 1012, 3143, 2240, 1017, 2917, 1012,  102]],
        device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
        device='cuda:0')}

In [99]:
encoded_ds["test"][0]

{'input_ids': tensor([  101,  2817,  1996,  2206, 29296,  4861,  1012,  2043,  2508, 15389,
          1996,  2682, 29296,  4861,  1010,  2002,  4858,  2008,  1996,  7809,
          5651,  2053,  3463,  1012,  2029,  1997,  1996,  2206,  2024,  2825,
          3114,  1006,  1055,  1007,  1029,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [97]:
logits = outputs.logits
logits.shape

torch.Size([1, 25])

In [98]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Algorithm Design', 'Program Development']
